### Build a Simple LLM Application with LCEL
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After seeing this video, you'll have a high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

In [1]:
### Open AI API Key and Open Source models--Llama3,Gemma2,mistral--Groq

import os
from dotenv import load_dotenv
load_dotenv()

groq_key=os.getenv("GROQ_API_KEY")

In [ ]:
from langchain_groq import ChatGroq
model=ChatGroq(model="openai/gpt-oss-120b", groq_api_key=groq_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000207EDC146E0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000207EDC15010>, model_name='openai/gpt-oss-120b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
from langchain_core.messages import HumanMessage,SystemMessage

messages=[
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello How are you?")
]

result=model.invoke(messages)

In [5]:
result

AIMessage(content='Bonjour, comment ça va\u202f?', additional_kwargs={'reasoning_content': 'We need to follow system instructions: we are ChatGPT, must obey OpenAI policy. The user asks to translate "Hello How are you?" into French. That\'s straightforward. Provide translation: "Bonjour, comment ça va ?" Possibly "Bonjour, comment allez-vous ?" depending formal. Provide translation.'}, response_metadata={'token_usage': {'completion_tokens': 76, 'prompt_tokens': 86, 'total_tokens': 162, 'completion_time': 0.152190828, 'prompt_time': 0.003419346, 'queue_time': 0.113281974, 'total_time': 0.155610174}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_3a688838c3', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--9171286b-5b7b-4bc0-9d62-bd53a5378ab8-0', usage_metadata={'input_tokens': 86, 'output_tokens': 76, 'total_tokens': 162})

In [6]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()
parser.invoke(result)

'Bonjour, comment ça va\u202f?'

In [7]:
### Using LCEL- chain the components
chain=model|parser
chain.invoke(messages)

'Bonjour, comment ça va\u202f?'

In [8]:
### Prompt Templates
from langchain_core.prompts import ChatPromptTemplate

generic_template="Trnaslate the following into {language}:"

prompt=ChatPromptTemplate.from_messages(
    [("system",generic_template),("user","{text}")]
)



In [9]:
result=prompt.invoke({"language":"French","text":"Hello"})

In [10]:
result.to_messages()

[SystemMessage(content='Trnaslate the following into French:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

In [11]:
##Chaining together components with LCEL
chain=prompt|model|parser
chain.invoke({"language":"French","text":"Hello"})

'**Bonjour**'